# Node Specification

@[Chaoming Wang](https://github.com/chaoming0625)

Neural networks in BrainPy are used to build dynamical systems. The [brainpy.nn](../apis/nn.rst) module provides various classes representing the nodes of a neural network. All of them are subclasses of the ``brainpy.nn.Node`` base class.

In [1]:
import brainpy as bp
import brainpy.math as bm

# bp.math.set_platform('cpu')

## What is a node?

In BrainPy, the ``Node`` instance is the basic element to form a network model. It is a unit on a graph, connected to other nodes by edges. 

In general, each ``Node`` instance in BrainPy has four components: 

- Feedforward inputs
- Feedback inputs
- State
- Output

It is worthy to note that each ``Node`` instance may have multiple feedforward or feedback connections. However, it only has one state and one output. 

<img src="../_static/node_specification.png"  width="50%" align="center" />

Each node has the following attributes:

- ``state``: the state of the node. It can be None if the node has no state to hold. 
- ``feedforward_shapes``: the shapes of the feedforward inputs. 
- ``feedback_shapes``: the shapes of the feedback inputs. 
- ``output_shape``: the output shape of the node. 


It also has several boolean attributes:

- ``trainable``: whether the node can be trained by ``brainpy.nn.RNNTrainer`` or other specified training methods. 
- ``is_ff_initialized``: whether the node has initialized the feedforward (input) connections. 
- ``is_fb_initialized``: whether the node has initialized the feedback (input) connections. 
- ``is_state_initialized``: whether the node state has been initialized. 


## Creating a node

A layer can be created as an instance of a ``brainpy.nn.Node`` subclass. For example, a dense layer can be created as follows:

In [2]:
bp.nn.Dense(num_unit=100) 

Dense(name=Dense0, trainable=True, forwards=None, feedbacks=None, 
      output=None, support_feedback=False, data_pass_type=PASS_SEQUENCE)

This will create a dense layer with 100 units.

Of course, if you have known the shapes of the feedforward connections, you can use ``input_shape``. 

In [3]:
bp.nn.Dense(num_unit=100, input_shape=128) 

Dense(name=Dense1, trainable=True, forwards=((None, 128),), feedbacks=None, 
      output=None, support_feedback=False, data_pass_type=PASS_SEQUENCE)

This create a densely connected layer which connected to another input layer with 128 dimension. 

## Naming a node

For convenience, you can name a layer by specifying the name keyword argument:

In [4]:
bp.nn.Dense(num_unit=100, input_shape=128, name='hidden_layer')

Dense(name=hidden_layer, trainable=True, forwards=((None, 128),), feedbacks=None, 
      output=None, support_feedback=False, data_pass_type=PASS_SEQUENCE)

## Initializing parameters

Many nodes have their parameters. We can set the parameter of a node will the following methods. 

- **Tensors**

If a tensor variable instance is provided, this is used unchanged as the parameter variable. For example:

In [5]:
l = bp.nn.Dense(num_unit=50, input_shape=10, 
                weight_initializer=bm.random.normal(0, 0.01, size=(10, 50)))
l.initialize(num_batch=1)

l.weights.shape

(10, 50)

- **Callable function**

If a callable function (which receives a ``shape`` argument) is provided, the callable will be called with the desired shape to generate suitable initial parameter values. The variable is then initialized with those values. For example:

In [6]:
def init(shape):
    return bm.random.random(shape)

l = bp.nn.Dense(num_unit=30, input_shape=20, weight_initializer=init)
l.initialize(num_batch=1)

l.weights.shape

(20, 30)

- **Instance of** ``brainpy.init.Initializer`` 

If a ``brainpy.init.Initializer`` instance is provided, the initial parameter values will be generated with the desired shape by using the Initializer instance. For example:

In [7]:
l = bp.nn.Dense(num_unit=100, input_shape=20, 
                weight_initializer=bp.init.Normal(0.01))
l.initialize(num_batch=1)

l.weights.shape

(20, 100)

The weight matrix $W$ of this dense layer will be initialized using samples from a normal distribution with standard deviation 0.01 (see [brainpy.initialize](../apis/auto/initialize.rst) for more information).

- **None parameter**

Some types of parameter variables can also be set to ``None`` at initialization (e.g. biases). In that case, the parameter variable will be omitted. For example, creating a dense layer without biases is done as follows:

In [8]:
l = bp.nn.Dense(num_unit=100, input_shape=20, bias_initializer=None)
l.initialize(num_batch=1)

print(l.bias)

None


## ``trainable`` settings

Setting the node to be trainable or un-trainable can be easily achieved. This is controlled by the ``trainable`` argument when initializing a node. 

For example, for a un-trainable dense layer, the *weights* and *bias* are JaxArray instances. 

In [9]:
l = bp.nn.Dense(num_unit=3, input_shape=4, trainable=False)
l.initialize(num_batch=1)

l.weights

JaxArray([[-0.12613049,  0.14836858,  0.7535165 ],
          [-0.539558  ,  0.12115271, -0.46093628],
          [-0.7500996 ,  0.01327358, -0.02536432],
          [-0.3918407 , -0.07949126,  0.2674435 ]], dtype=float32)

When creating a layer with trainable setting, ``TrainVar`` will be created for them and initialized automatically. For example:

In [10]:
l = bp.nn.Dense(num_unit=3, input_shape=4, trainable=True)
l.initialize(num_batch=1)

l.weights

TrainVar([[-0.24284273,  0.1005571 , -0.9604599 ],
          [ 0.09039249,  0.4681486 ,  0.666192  ],
          [ 0.24261971,  0.55806404, -0.30615446],
          [-0.99454516, -0.06997029, -0.04132903]], dtype=float32)